# Central Limit Theorem 

In this tutorial, we will be displaying benefits of using iPywidgets in a textbook setting. 
The current way of showing the Central Limit Theorem concept is by [overlaying histograms](https://www.inferentialthinking.com/chapters/12/4/central-limit-theorem.html) to show students that the distibution becomes skinnier as the number of samples increase. 

## Using Interact

In [22]:
#Data science modules
from datascience import *
import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np

# Interaction
from IPython.display import display
from functools import partial
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Here's a simple example using coin flips

In [23]:
coin = Table().with_column('Coin', np.arange(0,2))
coin

Coin
0
1


Using interact, we create a visualization of the sample growing as the sample size increases. The (0,20) value for sample_num creates a slider ranging from 0 to 20.

In [24]:
def coin_sample_10(sample_num=0):
    return coin.sample(sample_num)

_ = interact(coin_sample_10, sample_num=(0, 20))

A Jupyter Widget

This is a function that plots the means of multiple resample from the coin flip distibution. This example can be found in the textbook for a roulette wheel. 

In [25]:
def coin_clt_hist(repetitions, num_flips):
    net_gain_coin_flips = make_array()
    for i in np.arange(repetitions):
        new_coin= coin.sample(num_flips)
        new_net_gain_coin_flip = new_coin.column('Coin').mean()
        net_gain_coin_flips = np.append(net_gain_coin_flips, new_net_gain_coin_flip)
    results = Table().with_column(
    'Net Gain Coin', net_gain_coin_flips
    )
    results.hist()

By simply writing interact, we can create an interactive output that allows students to visualize the histogram for different number of coin flips per resample and how that affects the distribution. All you need to pass in is the starting value of the slider and the max/min for the range. 

In [26]:
_ = interact(coin_clt_hist, repetitions=fixed(100),
             num_flips=widgets.IntSlider(value=50, max=500))

A Jupyter Widget

## Breaking down interact

Here we will break down interact into individual widgets to show how interact is built. 

First we create the button seen above. This can be done using the widgets library and calling a button called the IntSlider and passing in the initial value of the slider as well as the max. 

In [27]:
int_slider = widgets.IntSlider(value=50, max=500)

We can display this button by passing in the button into the display function. 

In [28]:
display(int_slider)

A Jupyter Widget

We want to design a function that runs every time a widget is moved. This function uses a parameter labeled "change". Change is a dictionary consisting of the old and new value. For example, if initizlly the slider is at 10, by sliding it to 100, change['new'] would return 100. Change['old'] would return 10, the old value. 

In [29]:
def new_coin_clt_hist(change, repetitions=100 ):
    num_flips = change['new']
    net_gain_coin_flips = make_array()
    for i in np.arange(repetitions):
        new_coin= coin.sample(num_flips)
        new_net_gain_coin_flip = new_coin.column('Coin').mean()
        net_gain_coin_flips = np.append(net_gain_coin_flips, new_net_gain_coin_flip)
    results = Table().with_column(
    'Net Gain Coin', net_gain_coin_flips
    )
    results.hist()


Here we link the button to the coin_clt_hist function by using the function observe and we pass in the name of whatever we want to send to the function, which would be the 'value' of the button in this case.  

In [30]:
int_slider.observe(new_coin_clt_hist, names='value')
display(int_slider)


A Jupyter Widget

## Adding Plotly

Now, adding plotly into the equation is not a simple step. However it can generate graphs that update smoothly and can be helpful when creating more complicated graphs. 

In [31]:
#plotly packages
import plotly
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.widgets import GraphWidget

This is also a similar function to before except this time the function returns the means rather than plotting them right away. 

In [32]:
def plotly_coin_clt_hist(change, repetitions=100 ):
    num_flips = change['new']
    net_gain_coin_flips = make_array()
    for i in np.arange(repetitions):
        new_coin= coin.sample(num_flips)
        new_net_gain_coin_flip = new_coin.column('Coin').mean()
        net_gain_coin_flips = np.append(net_gain_coin_flips, new_net_gain_coin_flip)
    return net_gain_coin_flips

Here we initialize a GraphWidget which is a Plotly wrapper around iPython widgets. Here the url links to a blank graph that we will update on. We can treat it as a template to build our new graph on top of. 

In [33]:
g = GraphWidget('https://plot.ly/~calebs11/86')

This is just an IntWidget identical to the one we created earlier. 

In [34]:
style = {'description_width': 'initial'}
num_flips_slider = widgets.IntSlider(
    description='Num Coin Flps',
    value=500, 
    max=1000, 
    min=1, 
    style=style, )

This is the trickiest part about using plotly. Rather than simply writing an update function that replots the graph each time the slider is moved, we are updating the input to our graph based on the value of the slider. The function restyle passes in the new input based on the new value of the slider. 

In [38]:
def update(change):
    new_sample = plotly_coin_clt_hist(change)
    g.restyle({'x': [new_sample], 'name': 'Coin Flip Average', 'type': 'histogram'})
    g.relayout({'xaxis.range': [0, 1], 'xaxis.fixedrange': True, 'yaxis.fixedrange': True})


We have to link our slider to the function, similar to before. 

In [39]:
num_flips_slider.unobserve_all()
num_flips_slider.observe(update, names='value')

Now just call display!

In [40]:
display(num_flips_slider)
display(g)

A Jupyter Widget

A Jupyter Widget